In [ ]:
#https://www.kaggle.com/kazanova/sentiment140 - another dataset
import numpy as np
import itertools
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn import feature_extraction
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cufflinks as cf
import plotly.offline as offline
from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix
cf.go_offline()
offline.init_notebook_mode()
from collections import Counter
# from sklearn import metrics #cross_validation,   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# train = pd.read_pickle('test_clean.pkl')
# test = pd.read_pickle('train_clean.pkl')
train = pd.read_pickle('test_clean_old.pkl')
test = pd.read_pickle('train_clean_old.pkl')

In [ ]:
# value_countrs_df = test['SentimentText'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).sort_values(ascending=False)
# value_countrs_df[:50].iplot(kind='bar')
counter = Counter()
stoplist = stopwords.words('english')
for index, row in test.iterrows():
    for word in row['SentimentText'].split():
        if word not in stoplist:
            counter[word] += 1
# counter.most_common(30)

In [ ]:
counter_df = pd.DataFrame.from_dict(counter, orient='index').sort_values(by=0, ascending=False)
counter_df.iloc[:30].iplot(kind='bar')

In [ ]:
counter_df.shape

In [ ]:
vectorizer = CountVectorizer(stop_words='english') # token_pattern can be provided to clarify tags
# Here we received a sparse matrix of 1 and 0s
# tfidf_transformer = TfidfTransformer()
# vectorized_text = tfidf_transformer.fit_transform(vectorizer.fit_transform(test['SentimentText'].values))
# an article for TF/IDF: https://medium.freecodecamp.org/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3
vectorized_text = vectorizer.fit_transform(test['SentimentText'].values)

In [ ]:
from IPython.display import Image
Image("count_vectorizer_tf_idf.jpeg")

In [ ]:
test.head(10) # Preprocessing/feature extraction/feature engineering process can be improved, other features can be encoded with one hot encoding
# label encoding

In [ ]:
X = vectorized_text
y = test['Sentiment']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, train_size=.8, stratify=y)

In [ ]:
# Todo add grid search or smth like that to optimize
# An article for the explanation: https://xgboost.readthedocs.io/en/latest/tutorials/model.html
# Random forest trees explanation: https://towardsdatascience.com/enchanted-random-forest-b08d418cb411
model = XGBClassifier(n_estimators=150, learning_rate=0.3, max_depth=20)
model.fit(x_train, y_train)

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
class_names = y.unique()
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

In [ ]:
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# https://www.kdnuggets.com/2016/06/regularization-logistic-regression.html
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html + regularization

In [ ]:
param_test1 = {
 'max_depth': np.arange(3,30,2),
 'min_child_weight': np.arange(1,6,2),
 'n_estimators': np.arange(90,170,10),
 'learning_rate': np.arange(0.1, 0.4, 0.1),
}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective='binary:logistic', scale_pos_weight=1, seed=27), 
 param_grid=param_test1, scoring='roc_auc',n_jobs=-1, iid=False, cv=5)
gsearch.fit(x_train, y_train)

In [ ]:
gsearch.best_score_, gsearch.best_params_, gsearch.best_score_, gsearch.score

(0.7932173620974299,
 {'learning_rate': 0.2,
  'max_depth': 29,
  'min_child_weight': 1,
  'n_estimators': 150},
 0.7932173620974299,
 <bound method BaseSearchCV.score of GridSearchCV(cv=5, error_score='raise',
        estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=5, min_child_weight=1, missing=None, n_estimators=140,
        n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
        subsample=0.8),
        fit_params=None, iid=False, n_jobs=-1,
        param_grid={'max_depth': range(3, 30, 2), 'learning_rate': array([0.1, 0.2, 0.3, 0.4]), 'min_child_weight': range(1, 6, 2), 'n_estimators': range(90, 170, 10)},
        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
        scoring='roc_auc', verbose=0)>)

## Check if we have overfit on the test dataset
Another approach is to have 3 datasets initially, train, test, validate

In [ ]:
from sklearn.model_selection import cross_val_score
clf = XGBClassifier(learning_rate=0.2, n_estimators=150, max_depth=100,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective='binary:logistic', scale_pos_weight=1, seed=270)
# model.fit(x_train, y_train)

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# K-fold cross-validation https://machinelearningmastery.com/k-fold-cross-validation/
# Add a validation dataset not to have an overfit on test.

In [ ]:
grid_predictions = gsearch.predict(x_test)
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))
# https://machinelearningmastery.com/tune-number-size-decision-trees-xgboost-python/

In [ ]:
Image("metrics.png")
# support - is number of sampels per class
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [ ]:
model_full_dataset = XGBClassifier()
model_full_dataset.fit(X, y)

In [ ]:
vectorized_text_train = tfidf_transformer.transform(vectorizer.transform(train['SentimentText'].as_matrix()))
# y_pred_full_dataset = model.predict(vectorized_text_train)


In [ ]:
predicted_full_dataset = model_full_dataset.predict(vectorized_text_train)

Other topics which can be covered:
- Compute lenvinstein distance
- Clusterize
- Add search by twitters similar to 
- Dimentionality reduction, e.g. https://github.com/lmcinnes/umap
- Add validation dataset to prevent overfitting or K-fold validation